## 3. 그룹 내 행 순서 함수
### 3-1 FIRST_VALUE 함수
파티션별 윈도우에서 가장 먼저 나온 값을 구하며 MIN함수를 활용하여 같은 결과를 얻을 수도 있다.   
    → 다른 함수와 달리 공동 등수를 인정하지 않고 처음 나온 행만을 처리한다.

In [ ]:
SELECT DEPTNO, ENAME, SAL, 
    FIRST_VALUE(ENAME) OVER (PARTITION BY DEPTNO ORDER BY SAL DESC ROWS UNBOUNDED PRECEDING) as DEPT_RICH
FROM EMP;

＊ROWS UNBOUNDED PRECEDING : 현재 행을 기준으로 파티션 내의 첫 번째 행까지의 범위를 지정한다.

> 같은 부서별로 PARTITION 생성한 뒤, 직원들을 SAL을 기준으로 내림차순 정렬하고,    
> 해당하는 PARTITION 내부에서 가장 첫번째로 출력된(SAL이 가장 높은) ROW의 ENAME을 DEPT_RICH 칼럼에 출력함!

### 3-2 LAST_VALUE 함수
파티션별 윈도우에서 가장 나중에 나온 값을 구한다. MAX함수를 활용하여 같은 결과를 얻을 수도 있다.   
    → FIRST_VALUE 함수와 동일하게 공동 등수를 인정하지 않고 제일 마지막에 나온 행만을 처리한다.


In [ ]:
SELECT DEPTNO, ENAME, SAL, 
    LAST_VALUE(ENAME) OVER (PARTITION BY DEPTNO ORDER BY SAL DESC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) as DEPT_POOR
FROM EMP;

＊ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING : 현재 행을 포함해서 파티션 내의 마지막 행까지의 범위를 지정한다.

> 같은 부서별로 PARTITION 생성한 뒤, 직원들을 SAL을 기준으로 내림차순 정렬하고,   
> 해당하는 PARTITION 내부에서 가장 마지막으로 출력된(SAL이 가장 낮은)ROW의 ENAME을 DEPT_POOR 칼럼에 출력함! 

---
**FIRST_VALUE와 LAST_VALUE 공통사항!**

＊ 공동 등수가 존재하는 경우 별도의 정렬 조건을 가진 INLINE VIEW를 사용하거나 ORDER BY 절에 칼럼을 추가해야 한다.

SELECT DEPTNO, ENAME, SAL,   
    FIRST_VALUE(ENAME) OVER (PARTITION BY DEPTNO ORDER BY SAL DESC **, ENAME ASC** ROWS UNBOUNDED PRECEDING) as RICH_EMP

---

### 3-3 LAG 함수
파티션별 윈도우에서 이전 몇 번째 행의 값을 가져올 수 있다.

#### LAG 함수의 ARGUMENTS (COLUMN, NUMBER_1, NUMBER_2)
- NUMBER_1 : 몇 번째 앞의 행을 가져올지 결정 (DEFAULT = 1)
- NUMBER_2 : 가져올 데이터가 없어 NULL값이 들여올 경우 대체될 값

In [ ]:
SELECT ENAME, HIREDATE, SAL, LAG(SAL) OVER (ORDER BY HIREDATE) as PREV_SAL
FROM EMP
WHERE JOB = 'SALESMAN';

> 직원들을 입사가 빠른 순으로 정렬하고, 본인보다 입사 일자가 한명 앞선 사원의 급여를 본인의 급여와 함께 출력한다.

In [ ]:
SELECT ENAME, HIREDATE, SAL, LAG(SAL, 2, 0) OVER (ORDER BY HIREDATE) as PREV_SAL
FROM EMP
WHERE JOB = 'SALESMAN';

> 위의 예시와 동일하게 동작하지만 본인보다 입사 일자가 2명 앞선 사원의 급여를 본인의 급여와 함께 출력하며,   
> 맨 처음으로 출력되는 2명은 가져올 데이터가 없기 떄문에 '0'으로 대체된다.

### 3-4 LEAD 함수
파티션별 윈도우에서 이후 몇 번째 행의 값을 가져올 수 있다. ( LAG 함수와 정 반대의 기능 수행)

#### LEAD 함수의 ARGUMENTS (COLUMN, NUMBER_1, NUMBER_2)
- NUMBER_1 : 몇 번째 뒤의 행을 가져올지 결정(DEFAULT = 1)
- NUMBER_2 : 가져올 데이터가 없어 NULL 값이 들여올 경우 대체될 값

In [ ]:
SELECT ENAME, HIREDATE, LEAD(HIREDATE,1) OVER (ORDER BY HIREDATE) as 'NEXTHIRED'
FROM EMP;

> 직원들을 입사가 빠른 순으로 정렬하고, 본인보다 입사 일자가 한명 늦은 사원의 입사일자를 본인의 입사일자와 함께 줄력한다.   
> 별도로 대체될 값을 지정하지 않은 상태이므로, 가장 마지막에 출력되는 ROW의 NEXTHIRED COLUMN의 값은 비어있다.

## 4. 그룹 내 비율 함수
### 4-1 RATIO_TO_REPORT 함수
파티션 내 전체 SUM(칼럼)값에 대한 행별 칼럼 값의 백분율을 소수점으로 구할 수 있다.   
결과 값은 **0 < X <= 1** 의 범위를 가진다.
    ＊개별 RATIO의 합 = 1

In [ ]:
SELECT ENAME, SAL, ROUND(RATIO_TO_REPORT(SAL) OVER (), 2) as P_R
FROM EMP
WHERE JOB='SALESMAN';

> RATIO_TO_REPORT를 활용하여 전체 값 중 해당 행의 SAL 값이 차지하는 비율을 P_R COLUMN에 값으로 리턴하는데,    
> ROUND 함수를 통해 소수 둘째자리까지 표현하였다.

### 4-2 PERCENT_RANK 함수
파티션별 윈도우에서 제일 먼저 나오는 것을 0으로, 제일 늦게 나오는 것을 1로 하여 값이 아닌 행의 순서별 백분율을 구한다.   
결과값은 **0 <= X <= 1**의 범위를 가진다.

In [ ]:
SELECT DEPTNO, ENAME, SAL, PERCENT_RANK() OVER (PARTITION BY DEPTNO ORDER BY SAL DESC) as P_R
FROM EMP;

> 부서를 기준으로 PARTITION한 그룹 내부에서 SAL을 기준으로 내림차순 정렬을 한다.   
> 정렬된 결과를 기준으로 해당 ROW의 위치를 PERCENT_RANK를 활용하여 상위 몇%의 등수 위치인지 P_R COLUMN의 값으로 리턴한다.


PERCENT_RANK를 수행하려는 COLUMN이 다른 행과 동일한 값을 가질 경우 : 1위(공동):0 / 1위(공동):0 / 3위:0.5 / 4위:0.75 / 5위:1

### 4-3 CUME_DIST 함수
파티션별 윈도우의 전체 건수에서 현재 행보다 작거나 같은 건수에 대한 누적 백분율을 구한다.   
결과값은 **0 < X <= 1**의 범위를 가진다.

In [ ]:
SELECT DEPTNO, ENAME, SAL, CUME_DIST() OVER (PARTITION BY DEPTNO ORDER BY SAL DESC) as CUME_DIST
FROM EMP;

> 부서를 기준으로 PARTITION한 그룹 내부에서 SAL을 기준으로 내림차순 정렬을 한다.   
> 정렬된 결과를 기준으로 해당 ROW의 위치를 CUME_DIST를 활용하여 상위 몇%인지 CUME_DIST COLUMN의 값으로 리턴한다.

다른 WINDOW 함수의 경우 동일한 값으로 같은 순서이면 앞의 결과를 기준으로 동일하게 사용하지만, **CUME_DIST는 뒤의 결과를 기준으로 한다.**    
예시) 1위:0.2 / 2위:0.4 / **3위(공동):0.8 / 3위(공동):0.8** / 5위:1

### 4-4 NTILE 함수
파티션별 전체 건수를 ARGUMENT 값으로 N등분한 결과를 구할 수 있다.

In [ ]:
SELECT ENAME, SAL, NTILE(4), OVER (ORDER BY SAL DESC) as QUAR_TILE
FROM EMP;

> 급여를 기준으로 전체 사원을 정렬한 뒤 4개의 그룹으로 나누어 순서대로 1,2,3,4의 값을 QUAR_TILE COLUMN에 리턴한다.

#### ROW의 총 갯수 X를 N등분 했을 때 나머지는 제일 앞 그룹부터 1개씩 추가로 분배한다.

4명 직원의 경우 - A:1 / B:2 / C:3 / D:4

5명 직원의 경우 - A,B:1 / C:2 / D:3 / E:4

6명 직원의 경우 - A,B:1 / C,D:2 / E:3 / F:4
